In [2]:
!conda install -c conda-forge folium=0.5.0

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis and data manipulation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
CLIENT_ID = 'CLUNPRG0MTBOITNDUVNWU5TLCKHEPVYC52W3HVWUAFJIMXJP' # your Foursquare ID
CLIENT_SECRET = 'X4YDBEWY3Y5JSIVIUCK01LHITEKWCBNAA5ZQO3FWZ0CO3HXR' # your Foursquare Secret
VERSION = '20200301' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: CLUNPRG0MTBOITNDUVNWU5TLCKHEPVYC52W3HVWUAFJIMXJP


In [5]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # PIZZA PLACE CATEGORY ID
    results[city] = requests.get(url).json()

In [6]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

#### The FourSquare API limits us to the 100 nearest pizza venues in the city.
#### I need to check the pizza place densities by examination.

In [7]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of pizza places in New York, NY =  305
Showing Top 100
Total number of pizza places in Chicago, IL =  225
Showing Top 100
Total number of pizza places in San Francisco, CA =  169
Showing Top 100
Total number of pizza places in Jersey City, NJ =  124
Showing Top 100
Total number of pizza places in Boston, MA =  184
Showing Top 100


In [8]:
maps[cities[0]]

In [9]:
maps[cities[1]]


In [10]:
maps[cities[2]]

In [11]:
maps[cities[3]]

In [12]:
maps[cities[4]]

### Through examination of the FourSquare data and map illustration of pizza place density within the 5 cities. I can conclude that New York City and Jersey City have the highest density of pizza places within the five major cities I examined. 

### In order to quantitavely measure this density, I will be using statistical methods. First, I will get the mean location of pizza places which should be close to most locations and if they are  either really dense or far if not. Lastly, I will take the average of the distance of the pizza venues to the mean coordinates.



In [13]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.02222289381011469
Chicago, IL
Mean Distance from Mean coordinates
0.0589039075521342
San Francisco, CA
Mean Distance from Mean coordinates
0.028626658072997885
Jersey City, NJ
Mean Distance from Mean coordinates
0.019445551195654404
Boston, MA
Mean Distance from Mean coordinates
0.03574459449909179


In [14]:
maps[cities[0]]

In [15]:
maps[cities[1]]

In [16]:
maps[cities[2]]

In [17]:
maps[cities[3]]

In [18]:
maps[cities[4]]

### Through examination of the mean distance of pizza places from the mean coordinates. Regardless, of the number of pizza places of a given city, I can conclude that New York City has the highest denisty of pizza places with a small radius. Chicago and Boston on the other hand have their pizza places spread apart more and this is illustrated through their mean distance from the mean coordinates. Thus, my pizza connoisseur  client would make the most of his time reviewing pizza places in New York City. Due to the high density of pizza places in New York City than the other 4 cities, the client will be able to do more reviews of the pizza places in the short amount of time he has on his trip. Moreover, the clients best option would be to book a hotel near the mean coordinates of New York and this would give him access to the 100+ pizza stores that the city has to offer for reviewing.
